# PySpark Training Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


####  Run this cell to set up and start your interactive session.

In [5]:
%idle_timeout 60
%glue_version 5.0
%worker_type G.1X
%number_of_workers 2

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2


In [7]:
%%configure
{
    "--enable-continuous-cloudwatch-log": "true",
    "--enable-spark-ui": "true",
    "--spark-event-logs-path": "s3://dip-pyspark-training/spark_ui_tmp/",
    "--enable-metrics": "true",
    "--enable-observability-metrics": "true",
    "--conf": "spark.sql.codegen.comments=true",
    "--conf": "spark.sql.codegen.fallback=true",
    "--conf": "spark.sql.codegen.wholeStage=true",
    "--conf": "spark.sql.ui.explainMode=extended",
    "--conf": "spark.sql.ui.retainedExecutions=100",
    "--conf": "spark.ui.retainedJobs=1000",
    "--conf": "spark.ui.retainedStages=1000",
    "--conf": "spark.ui.retainedTasks=10000",
    "--conf": "spark.ui.showAdditionalMetrics=true"
}

The following configurations have been updated: {'--enable-continuous-cloudwatch-log': 'true', '--enable-spark-ui': 'true', '--spark-event-logs-path': 's3://dip-pyspark-training/spark_ui_tmp/', '--enable-metrics': 'true', '--enable-observability-metrics': 'true', '--conf': 'spark.ui.showAdditionalMetrics=true'}


#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 60
Session ID: 613a7f7e-0199-479e-9501-18fe0a567a11
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
--enable-continuous-cloudwatch-log true
--enable-spark-ui true
--spark-event-logs-path s3://dip-pyspark-training/spark_ui_tmp/
--enable-metrics true
--enable-observability-metrics true
--conf spark.ui.showAdditionalMetrics=true
Waiting for session 613a7f7e-0199-479e-9501-18fe0a567a11 to get into ready status...
Session 613a7f7e-0199-479e-9501-18fe0a567a11 has been created.



### Get spark's configuration

In [2]:
dynamic_allocation_enabled = spark.sparkContext.getConf().get('spark.dynamicAllocation.enabled')
dynamic_min_executors = spark.sparkContext.getConf().get('spark.dynamicAllocation.minExecutors')
dynamic_max_executors = spark.sparkContext.getConf().get('spark.dynamicAllocation.maxExecutors')
dynamic_initial_executors = spark.sparkContext.getConf().get('spark.dynamicAllocation.initialExecutors')

executor_instances = spark.sparkContext.getConf().get('spark.executor.instances')
executor_cores = spark.sparkContext.getConf().get('spark.executor.cores')
executor_memory = spark.sparkContext.getConf().get('spark.executor.memory')

driver_cores = spark.sparkContext.getConf().get('spark.driver.cores')
driver_memory = spark.sparkContext.getConf().get('spark.driver.memory')

print(f'''
Dynamic allocation enabled: {dynamic_allocation_enabled}
Dynamic min executors: {dynamic_min_executors}
Dynamic max executors: {dynamic_max_executors}
Dynamic initial executors: {dynamic_initial_executors}
----------------------------------------
Executor instances: {executor_instances}
Executor cores: {executor_cores}
Executor memory: {executor_memory}
----------------------------------------
Driver cores: {driver_cores}
Driver memory: {driver_memory}
''')


Dynamic allocation enabled: false
Dynamic min executors: 1
Dynamic max executors: 1
Dynamic initial executors: 3
----------------------------------------
Executor instances: 1
Executor cores: 4
Executor memory: 10g
----------------------------------------
Driver cores: 4
Driver memory: 10g


In [18]:
spark.sparkContext.getConf().getAll()

[('spark.blacklist.decommissioning.enabled', 'true'), ('spark.metrics.conf.*.source.jvm.class', 'org.apache.spark.metrics.source.JvmSource'), ('spark.executor.extraJavaOptions', "-Djava.net.preferIPv6Addresses=false -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+UseParallelGC -XX:InitiatingHeapOccupancyPercent=70 -XX:OnOutOfMemoryError='kill -9 %p' -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.

### Import libraries

In [3]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
import datetime

### Loading data

In [4]:
c_df = spark.read.format('parquet').load('s3://dip-pyspark-training/data/small/customers/')
c_df.rdd.getNumPartitions()

1


In [5]:
c_df.schema

StructType([StructField('cust_id', StringType(), True), StructField('name', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('birthday', StringType(), True), StructField('zip', StringType(), True), StructField('city', StringType(), True)])


In [6]:
c_df.show()

+----------+--------------+---+------+----------+-----+------------+
|   cust_id|          name|age|gender|  birthday|  zip|        city|
+----------+--------------+---+------+----------+-----+------------+
|C007YEYTX9|  Aaron Abbott| 34|Female| 7/13/1991|97823|      boston|
|C00B971T1J|  Aaron Austin| 37|Female|12/16/2004|30332|     chicago|
|C00WRSJF1Q|  Aaron Barnes| 29|Female| 3/11/1977|23451|      denver|
|C01AZWQMF3| Aaron Barrett| 31|  Male|  7/9/1998|46613| los_angeles|
|C01BKUFRHA|  Aaron Becker| 54|  Male|11/24/1979|40284|   san_diego|
|C01RGUNJV9|    Aaron Bell| 24|Female| 8/16/1968|86331|      denver|
|C01USDV4EE|   Aaron Blair| 35|Female|  9/9/1974|80078|    new_york|
|C01WMZQ7PN|   Aaron Brady| 51|Female| 8/20/1994|52204|philadelphia|
|C021567NJZ|  Aaron Briggs| 57|  Male| 3/10/1990|22008|philadelphia|
|C023M6MKR3|   Aaron Bryan| 29|  Male| 4/10/1976|05915|philadelphia|
|C0248N0EK3|  Aaron Burton| 26|Female| 8/27/1964|50477| los_angeles|
|C02C54RPNL|  Aaron Burton| 46|  M

In [7]:
t_df = spark.read.format('parquet').load('s3://dip-pyspark-training/data/small/transactions/')
t_df.rdd.getNumPartitions()

13


In [8]:
t_df.show()

+----------+----------+----------+---------------+----------+----+-----+---+-------------+-----+-------------+
|   cust_id|start_date|  end_date|         txn_id|      date|year|month|day| expense_type|  amt|         city|
+----------+----------+----------+---------------+----------+----+-----+---+-------------+-----+-------------+
|C0YDPQWPBJ|2010-12-01|2020-11-01|TM2DBM2C4ZRGDMK|2020-09-05|2020|    9|  5|Entertainment| 4.23| philadelphia|
|C0YDPQWPBJ|2010-12-01|2020-11-01|T9OJV04D72A2TZY|2017-08-16|2017|    8| 16|Entertainment| 6.51|       denver|
|C0YDPQWPBJ|2010-12-01|2020-11-01|TURTALJ1D3ESXWO|2016-12-19|2016|   12| 19| Motor/Travel|30.06|     new_york|
|C0YDPQWPBJ|2010-12-01|2020-11-01|TCWR5KX1P7KXH8X|2019-10-07|2019|   10|  7|Entertainment| 7.01|      chicago|
|C0YDPQWPBJ|2010-12-01|2020-11-01|TBFPYQ01GHQ2M4K|2013-04-17|2013|    4| 17|Entertainment| 8.64| philadelphia|
|C0YDPQWPBJ|2010-12-01|2020-11-01|T2YCJ2W1ZU1LGIM|2014-05-15|2014|    5| 15|Entertainment| 5.01|      seattle|
|

### Examples of narrow transformations

In [9]:
tmp_01_df = c_df.withColumn('first_name', F.split('name', ' ').getItem(0))

In [10]:
tmp_02_df = tmp_01_df.withColumn('last_name', F.split('name', ' ').getItem(1))

In [11]:
tmp_03_df = tmp_02_df.select(['cust_id', 'first_name', 'last_name', 'city', 'gender', 'birthday'])

In [12]:
tmp_04_df = tmp_03_df.filter(F.col('city') == 'chicago')

In [13]:
tmp_04_df.show()

+----------+----------+---------+-------+------+----------+
|   cust_id|first_name|last_name|   city|gender|  birthday|
+----------+----------+---------+-------+------+----------+
|C00B971T1J|     Aaron|   Austin|chicago|Female|12/16/2004|
|C034RB2MQ6|     Aaron|     Ford|chicago|  Male|  7/8/1988|
|C08RJGXUDH|     Aaron|  Jimenez|chicago|Female| 2/26/1982|
|C0EF3QYJVK|     Aaron|   Thomas|chicago|Female| 4/27/1974|
|C0FSON7A4J|     Abbie|    Burke|chicago|  Male| 3/28/1976|
|C0HCX8JU7B|     Abbie|  Edwards|chicago|  Male| 2/21/1998|
|C0LF7GK36H|     Abbie|    Lucas|chicago|  Male| 4/29/1961|
|C0VK4CF81S|       Ada|    Casey|chicago|  Male| 6/14/1972|
|C0VTJ1GFVP|       Ada|   Castro|chicago|  Male| 1/23/1969|
|C107K4GY1L|       Ada|     Reed|chicago|  Male| 1/28/1987|
|C12G5ORP07|       Ada|   Torres|chicago|Female|10/24/1964|
|C13UV4JLH1|       Ada|    Woods|chicago|Female| 4/22/2000|
|C146M4WYYT|      Adam| Alvarado|chicago|Female| 6/17/1980|
|C16UWLYOBC|      Adam|     Bush|chicago

In [14]:
tmp_04_df.explain(True)

== Parsed Logical Plan ==
'Filter ('city = chicago)
+- Project [cust_id#0, first_name#112, last_name#121, city#6, gender#3, birthday#4]
   +- Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, first_name#112, split(name#1,  , -1)[1] AS last_name#121]
      +- Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, split(name#1,  , -1)[0] AS first_name#112]
         +- Relation [cust_id#0,name#1,age#2,gender#3,birthday#4,zip#5,city#6] parquet

== Analyzed Logical Plan ==
cust_id: string, first_name: string, last_name: string, city: string, gender: string, birthday: string
Filter (city#6 = chicago)
+- Project [cust_id#0, first_name#112, last_name#121, city#6, gender#3, birthday#4]
   +- Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, first_name#112, split(name#1,  , -1)[1] AS last_name#121]
      +- Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, split(name#1,  , -1)[0] AS first_name#112]
         +- Rel

#### Parsed plan
```bash
== Parsed Logical Plan ==
'Project ['cust_id, 'first_name, 'last_name, 'city, 'gender, 'birthday]
+- Project [cust_id#88, name#89, age#90, gender#91, birthday#92, zip#93, city#94, first_name#579, split(name#89,  , -1)[1] AS last_name#588]
   +- Project [cust_id#88, name#89, age#90, gender#91, birthday#92, zip#93, city#94, split(name#89,  , -1)[0] AS first_name#579]
      +- Filter (city#94 = chicago)
         +- Relation [cust_id#88,name#89,age#90,gender#91,birthday#92,zip#93,city#94] parquet
```
#### Logical plan
```bash
== Analyzed Logical Plan ==
cust_id: string, first_name: string, last_name: string, city: string, gender: string, birthday: string
Project [cust_id#88, first_name#579, last_name#588, city#94, gender#91, birthday#92]
+- Project [cust_id#88, name#89, age#90, gender#91, birthday#92, zip#93, city#94, first_name#579, split(name#89,  , -1)[1] AS last_name#588]
   +- Project [cust_id#88, name#89, age#90, gender#91, birthday#92, zip#93, city#94, split(name#89,  , -1)[0] AS first_name#579]
      +- Filter (city#94 = chicago)
         +- Relation [cust_id#88,name#89,age#90,gender#91,birthday#92,zip#93,city#94] parquet
```
#### Optimized plan
```bash
== Optimized Logical Plan ==
Project [cust_id#88, split(name#89,  , -1)[0] AS first_name#579, split(name#89,  , -1)[1] AS last_name#588, city#94, gender#91, birthday#92]
+- Filter (isnotnull(city#94) AND (city#94 = chicago))
   +- Relation [cust_id#88,name#89,age#90,gender#91,birthday#92,zip#93,city#94] parquet
```
#### Physical plan
```bash
== Physical Plan ==
*(1) Project [cust_id#88, split(name#89,  , -1)[0] AS first_name#579, split(name#89,  , -1)[1] AS last_name#588, city#94, gender#91, birthday#92]
+- *(1) Filter (isnotnull(city#94) AND (city#94 = chicago))
   +- *(1) ColumnarToRow
      +- FileScan parquet [cust_id#88,name#89,gender#91,birthday#92,city#94] Batched: true, DataFilters: [isnotnull(city#94), (city#94 = chicago)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[s3://dip-pyspark-training/data/small/customers], PartitionFilters: [], PushedFilters: [IsNotNull(city), EqualTo(city,chicago)], ReadSchema: struct<cust_id:string,name:string,gender:string,birthday:string,city:string>
```

### Examples of wide transformations

#### Repartition

In [37]:
t_df.rdd.getNumPartitions()

13


In [39]:
t_df.repartition(20).explain(True)

== Parsed Logical Plan ==
Repartition 20, true
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
Repartition 20, true
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Optimized Logical Plan ==
Repartition 20, true
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(20), REPARTITION_BY_NUM, [plan_id=297]
   +- FileScan parquet [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expens

In [15]:
t_df.repartition('city').explain(True)

== Parsed Logical Plan ==
'RepartitionByExpression ['city]
+- Relation [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#51,expense_type#52,amt#53,city#54] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
RepartitionByExpression [city#54]
+- Relation [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#51,expense_type#52,amt#53,city#54] parquet

== Optimized Logical Plan ==
RepartitionByExpression [city#54]
+- Relation [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#51,expense_type#52,amt#53,city#54] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange hashpartitioning(city#54, 4), REPARTITION_BY_COL, [plan_id=90]
   +- FileScan parquet [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#51,expense_

In [16]:
t_df.repartition('city').rdd.getNumPartitions()

4


In [17]:
t_df.repartition(2, 'city').explain(True)

== Parsed Logical Plan ==
'RepartitionByExpression ['city], 2
+- Relation [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#51,expense_type#52,amt#53,city#54] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
RepartitionByExpression [city#54], 2
+- Relation [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#51,expense_type#52,amt#53,city#54] parquet

== Optimized Logical Plan ==
RepartitionByExpression [city#54], 2
+- Relation [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#51,expense_type#52,amt#53,city#54] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange hashpartitioning(city#54, 2), REPARTITION_BY_NUM, [plan_id=125]
   +- FileScan parquet [cust_id#44,start_date#45,end_date#46,txn_id#47,date#48,year#49,month#50,day#5

#### Coalesce

In [45]:
t_df.rdd.getNumPartitions()

13


In [51]:
t_df.coalesce(4).rdd.getNumPartitions()

4


In [50]:
t_df.coalesce(4).explain(True)

== Parsed Logical Plan ==
Repartition 4, false
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
Repartition 4, false
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Optimized Logical Plan ==
Repartition 4, false
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Physical Plan ==
Coalesce 4
+- *(1) ColumnarToRow
   +- FileScan parquet [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] Batched: true, DataFilters: [], Format: Parquet,

In [52]:
t_df.coalesce(1).explain(True)

== Parsed Logical Plan ==
Repartition 1, false
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
Repartition 1, false
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Optimized Logical Plan ==
Repartition 1, false
+- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet

== Physical Plan ==
Coalesce 1
+- *(1) ColumnarToRow
   +- FileScan parquet [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] Batched: true, DataFilters: [], Format: Parquet,

#### Join

In [59]:
t_df.show()

+----------+----------+----------+---------------+----------+----+-----+---+-------------+-----+-------------+
|   cust_id|start_date|  end_date|         txn_id|      date|year|month|day| expense_type|  amt|         city|
+----------+----------+----------+---------------+----------+----+-----+---+-------------+-----+-------------+
|C0YDPQWPBJ|2010-12-01|2020-11-01|TM2DBM2C4ZRGDMK|2020-09-05|2020|    9|  5|Entertainment| 4.23| philadelphia|
|C0YDPQWPBJ|2010-12-01|2020-11-01|T9OJV04D72A2TZY|2017-08-16|2017|    8| 16|Entertainment| 6.51|       denver|
|C0YDPQWPBJ|2010-12-01|2020-11-01|TURTALJ1D3ESXWO|2016-12-19|2016|   12| 19| Motor/Travel|30.06|     new_york|
|C0YDPQWPBJ|2010-12-01|2020-11-01|TCWR5KX1P7KXH8X|2019-10-07|2019|   10|  7|Entertainment| 7.01|      chicago|
|C0YDPQWPBJ|2010-12-01|2020-11-01|TBFPYQ01GHQ2M4K|2013-04-17|2013|    4| 17|Entertainment| 8.64| philadelphia|
|C0YDPQWPBJ|2010-12-01|2020-11-01|T2YCJ2W1ZU1LGIM|2014-05-15|2014|    5| 15|Entertainment| 5.01|      seattle|
|

In [60]:
c_df.show()

+----------+--------------+---+------+----------+-----+------------+
|   cust_id|          name|age|gender|  birthday|  zip|        city|
+----------+--------------+---+------+----------+-----+------------+
|C007YEYTX9|  Aaron Abbott| 34|Female| 7/13/1991|97823|      boston|
|C00B971T1J|  Aaron Austin| 37|Female|12/16/2004|30332|     chicago|
|C00WRSJF1Q|  Aaron Barnes| 29|Female| 3/11/1977|23451|      denver|
|C01AZWQMF3| Aaron Barrett| 31|  Male|  7/9/1998|46613| los_angeles|
|C01BKUFRHA|  Aaron Becker| 54|  Male|11/24/1979|40284|   san_diego|
|C01RGUNJV9|    Aaron Bell| 24|Female| 8/16/1968|86331|      denver|
|C01USDV4EE|   Aaron Blair| 35|Female|  9/9/1974|80078|    new_york|
|C01WMZQ7PN|   Aaron Brady| 51|Female| 8/20/1994|52204|philadelphia|
|C021567NJZ|  Aaron Briggs| 57|  Male| 3/10/1990|22008|philadelphia|
|C023M6MKR3|   Aaron Bryan| 29|  Male| 4/10/1976|05915|philadelphia|
|C0248N0EK3|  Aaron Burton| 26|Female| 8/27/1964|50477| los_angeles|
|C02C54RPNL|  Aaron Burton| 46|  M

In [66]:
# TEMPORARY disable auto broadcast join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [67]:
joined_df = t_df.join(
    other=c_df,
    how='inner',
    on='cust_id')

In [ ]:
joined.show()

In [65]:
joined.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [cust_id])
:- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet
+- Relation [cust_id#88,name#89,age#90,gender#91,birthday#92,zip#93,city#94] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string, name: string, age: string, gender: string, birthday: string, zip: string, city: string
Project [cust_id#102, start_date#103, end_date#104, txn_id#105, date#106, year#107, month#108, day#109, expense_type#110, amt#111, city#112, name#89, age#90, gender#91, birthday#92, zip#93, city#94]
+- Join Inner, (cust_id#102 = cust_id#88)
   :- Relation [cust_id#102,start_date#103,end_date#104,txn_id#105,date#106,year#107,month#108,day#109,expense_type#110,amt#111,city#112] parquet
   +- Relation [cust_id#88,name#8

#### GroupBy

In [30]:
g_df = t_df.groupBy('cust_id').agg({'txn_id': 'count', 'amt': 'sum'})

In [31]:
g_df.explain(True)

== Parsed Logical Plan ==
'Aggregate ['cust_id], ['cust_id, count(txn_id#61) AS count(txn_id)#410L, unresolvedalias('sum(amt#67), None)]
+- Relation [cust_id#58,start_date#59,end_date#60,txn_id#61,date#62,year#63,month#64,day#65,expense_type#66,amt#67,city#68] parquet

== Analyzed Logical Plan ==
cust_id: string, count(txn_id): bigint, sum(amt): double
Aggregate [cust_id#58], [cust_id#58, count(txn_id#61) AS count(txn_id)#410L, sum(cast(amt#67 as double)) AS sum(amt)#412]
+- Relation [cust_id#58,start_date#59,end_date#60,txn_id#61,date#62,year#63,month#64,day#65,expense_type#66,amt#67,city#68] parquet

== Optimized Logical Plan ==
Aggregate [cust_id#58], [cust_id#58, count(txn_id#61) AS count(txn_id)#410L, sum(cast(amt#67 as double)) AS sum(amt)#412]
+- Project [cust_id#58, txn_id#61, amt#67]
   +- Relation [cust_id#58,start_date#59,end_date#60,txn_id#61,date#62,year#63,month#64,day#65,expense_type#66,amt#67,city#68] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- H

In [32]:
g_df.show()

+----------+-------------+------------------+
|   cust_id|count(txn_id)|          sum(amt)|
+----------+-------------+------------------+
|CEVULXC7UN|         7146| 488571.1299999981|
|CSN6AIYZG9|         7566|          760164.9|
|CFSO1WP6OE|         7547| 891046.2500000012|
|CWZQEDFVMJ|         7988| 836305.6000000015|
|CBBGI50QZ1|         7953|  964384.620000003|
|CDY1XBW4KD|         7485|1601169.1700000048|
|C8R3WPMPE4|         7351| 842854.5299999998|
|CBQNUAPTEE|         7678| 854887.0599999976|
|CM60NRREKK|         7598| 189735.2500000001|
|C2Q7HQMS29|         7184|132972.24999999994|
|CP2WGJ7O5J|         7324| 198217.0900000002|
|C4IN9KHAHZ|         7156|150539.09000000008|
|CKVSGFJRQY|         7309|1302532.3599999994|
|CAHGQXCG13|         7391| 792579.1599999988|
|CL66HEUWQP|         7604|  571821.799999999|
|CYVHSYX1C2|         7629| 955859.7599999976|
|CRZCTBH6C2|         7462| 620124.5200000014|
|C3KUDEN3KO|         7999| 1535998.699999997|
|C1QEUH9YHR|         7408| 618350.

In [26]:
t_df.schema

StructType([StructField('cust_id', StringType(), True), StructField('start_date', StringType(), True), StructField('end_date', StringType(), True), StructField('txn_id', StringType(), True), StructField('date', StringType(), True), StructField('year', StringType(), True), StructField('month', StringType(), True), StructField('day', StringType(), True), StructField('expense_type', StringType(), True), StructField('amt', StringType(), True), StructField('city', StringType(), True)])


In [34]:
t_df.select('cust_id').distinct().count()

2958


In [36]:
t_df.repartition(64, 'cust_id').groupBy('cust_id').agg({'txn_id': 'count', 'amt': 'sum'}).explain(True)

== Parsed Logical Plan ==
'Aggregate ['cust_id], ['cust_id, count(txn_id#61) AS count(txn_id)#488L, unresolvedalias('sum(amt#67), None)]
+- RepartitionByExpression [cust_id#58], 64
   +- Relation [cust_id#58,start_date#59,end_date#60,txn_id#61,date#62,year#63,month#64,day#65,expense_type#66,amt#67,city#68] parquet

== Analyzed Logical Plan ==
cust_id: string, count(txn_id): bigint, sum(amt): double
Aggregate [cust_id#58], [cust_id#58, count(txn_id#61) AS count(txn_id)#488L, sum(cast(amt#67 as double)) AS sum(amt)#490]
+- RepartitionByExpression [cust_id#58], 64
   +- Relation [cust_id#58,start_date#59,end_date#60,txn_id#61,date#62,year#63,month#64,day#65,expense_type#66,amt#67,city#68] parquet

== Optimized Logical Plan ==
Aggregate [cust_id#58], [cust_id#58, count(txn_id#61) AS count(txn_id)#488L, sum(cast(amt#67 as double)) AS sum(amt)#490]
+- RepartitionByExpression [cust_id#58], 64
   +- Project [cust_id#58, txn_id#61, amt#67]
      +- Relation [cust_id#58,start_date#59,end_date#60